# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
#df = pd.read_csv('users_behavior.csv')
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)

# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None) 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [6]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [7]:
df.duplicated().sum()

0

### Вывод

* Датасет состоит из 3214 строк и 5 столбцов;
* Пропусков нет;
* Дубликатов нет.

## Разбейте данные на выборки

Извлечем целевой признак и признаки для обучения:

In [8]:
features = df.drop(['is_ultra'], axis=1) # признаки
target = df['is_ultra'] # целевой признак

Так как спрятанной тестовой выборки нет, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. 

Исходные данные разобъем в соотношении 3:1:1:

In [9]:
features_train, features_part, target_train, target_part = train_test_split(
   features, target, test_size=0.4, random_state=12345)

In [10]:
features_valid, features_test, target_valid, target_test = train_test_split(
   features_part, target_part, test_size=0.5, random_state=12345)

Проверим размеры получившихся выборок:

In [11]:
print(features_train.shape)
print(target_train.shape)
print()
print(features_valid.shape)
print(target_valid.shape)
print()
print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)

(643, 4)
(643,)

(643, 4)
(643,)


### Вывод

Данные разбиты на 3 выборки (обучающая, тестовая и валидационная) в соотношении 60/20/20.

## Исследуйте модели

### Дерево решений

Создадим модель DecisionTreeClassifier и выберем лучшую модель в зависимости от глубины:

In [12]:
%%time
best_model_dt = None
best_result_dt = 0
best_depth_dt = 0
for depth_dt in range(1, 11):
    model_dt = DecisionTreeClassifier(random_state=2761, max_depth=depth_dt) # обучите модель с заданной глубиной дерева
    model_dt.fit(features_train, target_train) # обучите модель
    predictions_dt = model_dt.predict(features_valid) # получите предсказания модели
    result_dt = accuracy_score(target_valid, predictions_dt) # посчитайте качество модели
    if result_dt > best_result_dt:
        best_model_dt = model_dt
        best_result_dt = result_dt
        best_depth_dt = depth_dt
        
print("depth =", best_depth_dt)       
print("Accuracy лучшей модели на валидационной выборке:", best_result_dt)

depth = 9
Accuracy лучшей модели на валидационной выборке: 0.7884914463452566
CPU times: user 55.6 ms, sys: 203 µs, total: 55.8 ms
Wall time: 53.6 ms


### Случайный лес

Создадим модель RandomForestClassifier и выберем лучшую модель в зависимости от глубины и количества деревьев:

In [13]:
%%time
best_model_rf = None
best_result_rf = 0
best_est_rf = 0
best_depth_rf = 0
for est_rf in range(15, 26):
    for depth_rf in range (5, 11):
        model_rf = RandomForestClassifier(random_state=2761, n_estimators=est_rf, max_depth=depth_rf)
        model_rf.fit(features_train, target_train) 
        predictions_rf = model_rf.predict(features_valid) 
        result_rf = accuracy_score(target_valid, predictions_rf) 
        if result_rf > best_result_rf:
            best_model_rf = model_rf
            best_result_rf = result_rf
            best_est_rf = est_rf
            best_depth_rf = depth_rf
            
print("depth =", best_depth_rf,"estimators =", best_est_rf)       
print("Accuracy лучшей модели на валидационной выборке:", best_result_rf)

depth = 7 estimators = 25
Accuracy лучшей модели на валидационной выборке: 0.8040435458786936
CPU times: user 3.71 s, sys: 14 ms, total: 3.72 s
Wall time: 3.72 s


### Логистическая регрессия

Создадим модель LogisticRegression и проверим метрику качества: 

In [14]:
model_lr = LogisticRegression(random_state=2761)
model_lr.fit(features_train, target_train) 
result_lr = model_lr.score(features_valid, target_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result_lr)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


### Вывод

Построив 3 модели, получились следующие значения accuracy:
* Дерево решений с параметрами глубины 9 - точность 0.788
* Случайный лес с параметрами количества деревьев 25 и глубины 7 - точность 0.804
* Логистическая регрессия - точность 0.759

Лучшие результаты получились у случайного леса.

## Проверьте модель на тестовой выборке

In [15]:
predictions_dt_test = best_model_dt.predict(features_test)
result_dt_test = accuracy_score(target_test, predictions_dt_test)
print('Accuracy лучшей модели решающего леса на тестовой выборке',result_dt_test)

predictions_rf_test = best_model_rf.predict(features_test)
result_rf_test = accuracy_score(target_test, predictions_rf_test)
print('Accuracy лучшей модели случайного леса на тестовой выборке',result_rf_test)

predictions_lr_test = model_lr.predict(features_test)
result_lr_test = accuracy_score(target_test, predictions_lr_test)
print('Accuracy модели логистической регресиии на тестовой выборке',result_lr_test)

Accuracy лучшей модели решающего леса на тестовой выборке 0.7838258164852255
Accuracy лучшей модели случайного леса на тестовой выборке 0.7978227060653188
Accuracy модели логистической регресиии на тестовой выборке 0.7402799377916018


### Вывод

Модель случайного леса при проверке на тестовой выборке показывает точность 0.7978, но ее недостатком является низкая скорость.

Решающее дерево работает быстрее, но качество чуть ниже, 0.7838. 

Логистическая регрессия показала наименьшую точность  0.74, что меньше требуемой точности в 75%.

## (бонус) Проверьте модели на адекватность

### Sanity Check

Сравнив модель со случайной, можно оценить её адекватность, или проверить на вменяемость (англ. sanity check).

Допустим, у нас есть модель, которая предсказывает все объекты случайным образом. С вероятностью 50/50 ответ будет 0 или 1.

accuracy = 0.5 * 0.5(для угаданных 1) + 0.5 * 0.5(для угаданных 0) = 0.5

Наша модель показывает accuracy 0.7978, значит она лучше случайной.

### Precision и Recall

* Точность (англ. precision) можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными.
* Полнота (англ. recall) показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.
*  F-мера — среднее гармоническое precision и recall. F-мера достигает максимума при полноте и точности, равными единице, и близка к нулю, если один из аргументов близок к нулю.

In [ ]:
#predictions_rf_test = best_model_rf.predict(features_test)
#result_rf_test = accuracy_score(target_test, predictions_rf_test)

In [16]:
precision_rf = precision_score(target_test, predictions_rf_test)
recall_rf = recall_score(target_test, predictions_rf_test)
f_score_rf = f1_score(target_test, predictions_rf_test)

print('Accuracy лучшей модели случайного леса на тестовой выборке',result_rf_test)
print('Precision лучшей модели случайного леса на тестовой выборке',precision_rf)
print('Recall лучшей модели случайного леса на тестовой выборке',recall_rf)
print('F-мера =', f_score_rf)

Accuracy лучшей модели случайного леса на тестовой выборке 0.7978227060653188
Precision лучшей модели случайного леса на тестовой выборке 0.7703703703703704
Recall лучшей модели случайного леса на тестовой выборке 0.5123152709359606
F-мера = 0.6153846153846154


Модель получилось не очень высокого качества, но проверка на адекватность пройдена.

### DummyClassifier

Сравним нашу модель с DummyClassifier:

In [17]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_train, target_train)

0.6924273858921162

Наша модель получилось более точной (0.7978) 

### Вывод

Сравнив модель с:
* Случайной (точность 0.5)
* DummyClassifier (точность 0.692)
* И рассчитав F-меру (0.615),

можно утверждать, что модель прошла проверку на адекватность.